In [2]:
import os
import pandas

for i in os.listdir('archive'):
    if i.endswith('.csv'):
        df = pandas.read_csv('archive/'+i)
        print(df.head(5))


   id            name  latitude  longitude
0   1      Memphis TN   35.1174   -89.9711
1   2      Chicago IL   41.8369   -87.6847
2   3      Houston TX   29.7604   -95.3698
3   4  Los Angeles CA   34.0500  -118.2500
4   5  New Orleans LA   29.9500   -90.0667
      id  product_id                 created_at                    sold_at  \
0  67971       13844  2022-07-02 07:09:20+00:00  2022-07-24 06:33:20+00:00   
1  67972       13844  2023-12-20 03:28:00+00:00                        NaN   
2  67973       13844  2023-06-04 02:53:00+00:00                        NaN   
3  72863       13844  2021-10-16 22:58:52+00:00  2021-11-22 02:19:52+00:00   
4  72864       13844  2021-08-07 16:33:00+00:00                        NaN   

      cost product_category            product_name       product_brand  \
0  2.76804      Accessories  (ONE) 1 Satin Headband  Funny Girl Designs   
1  2.76804      Accessories  (ONE) 1 Satin Headband  Funny Girl Designs   
2  2.76804      Accessories  (ONE) 1 Satin Headb

In [8]:
f"{tuple(df.keys())}"

"('id', 'first_name', 'last_name', 'email', 'age', 'gender', 'state', 'street_address', 'postal_code', 'city', 'country', 'latitude', 'longitude', 'traffic_source', 'created_at')"

In [22]:
[str(dtype) for dtype in df.dtypes.to_list()]

['int64',
 'object',
 'object',
 'object',
 'int64',
 'object',
 'object',
 'object',
 'object',
 'object',
 'object',
 'float64',
 'float64',
 'object',
 'object']

In [23]:
dtype_map = {
    'object': '%s',
    'int64': '%s',
    'float64': '%s',
    'bool': '%s',
    'datetime64[ns]': '%s'
}


In [45]:
import os
import pandas as pd
import mysql.connector

def load_csv_folder_to_mysql(
    directory: str,
    host: str,
    user: str,
    password: str,
    database: str
):
    # map pandas dtypes to MySQL column types
    sql_type_map = {
        'object':          'TEXT',
        'int64':           'BIGINT',
        'float64':         'DOUBLE',
        'bool':            'BOOLEAN',
        'datetime64[ns]':  'DATETIME'
    }

    conn = mysql.connector.connect(
        host=host,
        user=user,
        password=password,
        database=database
    )
    cursor = conn.cursor()

    for fname in os.listdir(directory):
        if not fname.endswith('.csv'):
            continue

        table = os.path.splitext(fname)[0]
        df = pd.read_csv(os.path.join(directory, fname))

        # --- convert all NaN/NaT to None so they become SQL NULLs ---
        df = df.where(pd.notnull(df), None)

        # --- 1) CREATE TABLE IF NOT EXISTS ---
        cols_defs = []
        for col, dtype in df.dtypes.items():
            sql_type = sql_type_map.get(str(dtype), 'TEXT')
            cols_defs.append(f"`{col}` {sql_type}")
        create_sql = f"""
            CREATE TABLE IF NOT EXISTS `{table}` (
              {', '.join(cols_defs)}
            ) ENGINE=InnoDB;
        """
        cursor.execute(create_sql)

        # --- 2) Build a fixed placeholders string ---
        col_names = ", ".join(f"`{c}`" for c in df.columns)
        placeholders = ", ".join(["%s"] * len(df.columns))
        insert_sql = f"INSERT INTO `{table}` ({col_names}) VALUES ({placeholders})"

        # --- 3) Insert row-by-row, passing the data tuple separately ---
        for row in df.itertuples(index=False, name=None):
            cursor.execute(insert_sql, row)

    conn.commit()
    cursor.close()
    conn.close()


load_csv_folder_to_mysql(
    directory="archive",
    host="localhost",
    user="root",
    password="manik",
    database="ecommerce"
)
